In [147]:
import numpy as np
from sklearn.linear_model import RidgeCV, LassoCV, HuberRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.neural_network import MLPRegressor
import random
from sklearn.linear_model import Ridge, RidgeCV, LassoCV

In [148]:
# read traning data
df = pd.read_csv("training_data.csv",low_memory=False)

In [149]:
vehicle_make_description=df['Vehicle_Make_Description'].values
short_name=[]
for i in range(len(vehicle_make_description)):
    name=vehicle_make_description[i][:3]
    short_name.append(name)
vehicles_short_name = pd.DataFrame(data = short_name)
df['Vehicle_Make_Description'] = vehicles_short_name

In [150]:
df=df.dropna()



In [151]:
for col in list(df):
    print(col)
    print(df[col].unique())

PolicyNo
[164532941 164533241 164534633 ... 381735600 382057400 382162500]
Policy_Company
['Standard ' 'Preferred']
Policy_Installment_Term
[ 6 12]
Policy_Billing_Code
['Direct Billed to Insured' 'Premium Finance']
Policy_Method_Of_Payment
['Pre-paid' 'Installment']
Policy_Reinstatement_Fee_Indicator
['N' 'Y']
Policy_Zip_Code_Garaging_Location
['43046' 'Unknown' '43555' ... '52183' '14344' '43191']
Vehicle_Territory
[35 17 27 30 31 32 26 13 16 36 37 20 18 22 15 19]
Vehicle_Make_Year
[2004 1980 2005 1985 1991 2001 1995 1997 1998 1990 1986 1992 1996 1984
 2002 1988 2006 1999 2003 2000 1975 1993 1982 1994 2007 1978 1983 1987
 1977 1973 1989 1969 1976 1954 1979 1970 1955 1964 1981 1966 1972 1965
 1971 1961 1962 1967 1968 1974 1960 1952 1953 1940 1963 1959 1957 1950
 1928 1956 1947 1948 1951 1939 1958 2008 1949 1941 1934 1930 1932 1937
 1924 1935 1938 1931 1936 1946 1945 1929 1942 1927]
Vehicle_Make_Description
['BUI' 'CAD' 'PON' 'MER' 'GMC' 'FOR' 'CHE' 'NSS' 'DOD' 'LIN' 'MAZ' 'HON'
 'JAG' 

In [152]:
print("Unknown values:")
for col in list(df):
    a=df[(df[col] =="Unknown" )]
    if(len(a)!=0):
        print(col)
        print(len(a))
print("-1  values")
for col in list(df):
    a=df[(df[col] ==-1 )]
    if(len(a)!=0):
        print(col)
        print(len(a))
print("1M - 1M values:")
for col in list(df):
    a=df[(df[col] =='1M-1M  ')]
    if(len(a)!=0):
        print(col)
        print(len(a))
print("99 values:")
for col in list(df):
    a=df[(df[col] ==99)]
    if(len(a)!=0):
        print(col)
        print(len(a))

Unknown values:
Policy_Zip_Code_Garaging_Location
3402


C:\Users\dell\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Vehicle_Annual_Miles
407018
Vehicle_Anti_Theft_Device
3
Vehicle_Passive_Restraint
78
EEA_Policy_Zip_Code_3
3402
-1  values
Vehicle_New_Cost_Amount
400355
Vehicle_Symbol
25
Vehicle_Miles_To_Work
253930
Vehicle_Days_Per_Week_Driven
18
Vehicle_Med_Pay_Limit
93798
Vehicle_Physical_Damage_Limit
12
Vehicle_Comprehensive_Coverage_Limit
407105
Vehicle_Collision_Coverage_Deductible
160145
EEA_Policy_Tenure
40
1M - 1M values:
Vehicle_Bodily_Injury_Limit
42
99 values:
Vehicle_Number_Of_Drivers_Assigned
114430
Vehicle_Miles_To_Work
64
Driver_Minimum_Age
12
Driver_Maximum_Age
14
Annual_Premium
3


In [153]:
#Observing which columns have same value or most missing as they won't affect the target variable and then dropping them
dropping_columns=['PolicyNo','EEA_PolicyYear','Policy_Zip_Code_Garaging_Location','Vehicle_Annual_Miles','Vehicle_New_Cost_Amount','Vehicle_Comprehensive_Coverage_Limit']
df = df.drop(dropping_columns, axis=1)


In [154]:

df=df[(df['Vehicle_Anti_Theft_Device'] !="Unknown" )]
df=df[(df['Vehicle_Passive_Restraint'] !="Unknown" )]
df=df[(df['EEA_Policy_Zip_Code_3'] !="Unknown" )]
df=df[(df['Vehicle_Symbol'] !=-1 )]
df=df[(df['Vehicle_Bodily_Injury_Limit'] !='1M-1M  ' )]
df=df[(df['Driver_Maximum_Age']!=99)]
df=df[(df['Driver_Minimum_Age']!=99)]
df=df[(df.Driver_Maximum_Age != 0 )]
df=df[(df.Driver_Minimum_Age != 0 )]
df.shape

(403546, 63)

In [155]:
#-1 values Vehicle_Days_Per_Week_Driven,
df['Vehicle_Days_Per_Week_Driven'].replace(-1,np.NaN,inplace=True)
Vehicle_Days_Per_Week_Driven_Mode=df['Vehicle_Days_Per_Week_Driven'].mode()
df['Vehicle_Days_Per_Week_Driven']=df['Vehicle_Days_Per_Week_Driven'].fillna(float(Vehicle_Days_Per_Week_Driven_Mode))
df['Vehicle_Days_Per_Week_Driven'].unique()

array([5., 4., 6., 3., 1., 0., 2., 7., 8., 9.])

In [156]:
df['Vehicle_Miles_To_Work'].replace(-1,np.NaN,inplace=True)
Vehicle_Miles_To_Work_Mean=df['Vehicle_Miles_To_Work'].mean()
a=int(Vehicle_Miles_To_Work_Mean)
df['Vehicle_Miles_To_Work']=df['Vehicle_Miles_To_Work'].fillna(a)
df['Vehicle_Miles_To_Work'].unique()

array([10., 12.,  7., 21., 14.,  8.,  4.,  2., 13.,  6.,  5., 25., 15.,
        3.,  1., 42., 20., 35., 40., 30., 50., 11.,  9., 22., 16., 26.,
       17., 23., 18.,  0., 24., 27., 60., 31., 29., 32., 99., 45., 28.,
       38., 49., 19., 37., 48., 34., 36., 95., 39., 62., 43., 52., 46.,
       65., 90., 55., 80., 58., 47., 44., 51., 57., 70., 71., 56., 33.,
       83., 75., 61., 64., 41., 93., 54., 63., 53., 78., 67., 72., 98.,
       59.])

In [157]:
# # Vehicle_Days_Per_Week_Driven
df['Vehicle_Days_Per_Week_Driven'].replace(-1,np.NaN,inplace=True)
Vehicle_Days_Per_Week_Driven_Mode=df['Vehicle_Days_Per_Week_Driven'].mode()
df['Vehicle_Days_Per_Week_Driven']=df['Vehicle_Days_Per_Week_Driven'].fillna(float(Vehicle_Days_Per_Week_Driven_Mode))
df['Vehicle_Days_Per_Week_Driven'].unique()

array([5., 4., 6., 3., 1., 0., 2., 7., 8., 9.])

In [158]:
# Vehicle_Med_Pay_Limit
df['Vehicle_Med_Pay_Limit'].replace(-1,np.NaN,inplace=True)
Vehicle_Med_Pay_Limit=df['Vehicle_Med_Pay_Limit'].mean()
df['Vehicle_Med_Pay_Limit']=df['Vehicle_Med_Pay_Limit'].fillna(int(Vehicle_Med_Pay_Limit))
df['Vehicle_Med_Pay_Limit'].unique()

array([ 1000., 50000.,  5000.,  3476., 10000., 25000.,  2000.])

In [159]:
# Vehicle_Physical_Damage_Limit
df['Vehicle_Physical_Damage_Limit'].replace(-1,np.NaN,inplace=True)
Vehicle_Physical_Damage_Limit_Mean=df['Vehicle_Physical_Damage_Limit'].mean()
df['Vehicle_Physical_Damage_Limit']=df['Vehicle_Physical_Damage_Limit'].fillna(int(Vehicle_Physical_Damage_Limit_Mean))
df['Vehicle_Physical_Damage_Limit'].unique()

array([100000.,  35000.,  25000.,  50000., 250000., 500000.,  50033.])

In [160]:
# Vehicle_Number_Of_Drivers_Assigned
df['Vehicle_Number_Of_Drivers_Assigned'].replace(99,np.NaN,inplace=True)
Vehicle_Number_Of_Drivers_Assigned_Mode=df['Vehicle_Number_Of_Drivers_Assigned'].mean()
df['Vehicle_Number_Of_Drivers_Assigned']=df['Vehicle_Number_Of_Drivers_Assigned'].fillna(int(Vehicle_Number_Of_Drivers_Assigned_Mode))
df['Vehicle_Number_Of_Drivers_Assigned'].unique()

array([1., 2., 3., 4., 5., 8., 7., 6.])

In [161]:
# Vehicle_Collision_Coverage_Deductible
df['Vehicle_Collision_Coverage_Deductible'].replace(-1,np.NaN,inplace=True)
Vehicle_Collision_Coverage_Deductible_Mean=df['Vehicle_Collision_Coverage_Deductible'].mean()
df['Vehicle_Collision_Coverage_Deductible']=df['Vehicle_Collision_Coverage_Deductible'].fillna(int(Vehicle_Collision_Coverage_Deductible_Mean))
df['Vehicle_Collision_Coverage_Deductible'].unique()

array([ 500.,  100.,  250.,  485., 1000.,  200.,   50., 3000., 2000.])

In [162]:
# EEA_Policy_Tenure
df['EEA_Policy_Tenure'].replace(-1,np.NaN,inplace=True)
EEA_Policy_Tenure_Mean=df['EEA_Policy_Tenure'].mean()
df['EEA_Policy_Tenure']=df['EEA_Policy_Tenure'].fillna(round(EEA_Policy_Tenure_Mean,1))
df['EEA_Policy_Tenure'].unique()


array([22.7, 47.2, 46.7, 16.5, 46.3, 17. , 47.7, 16. , 46.6, 47.1, 16.1,
       16.6, 47.5, 16.8, 16.2, 16.7, 25.3, 16.3, 37.5, 42. , 42.5, 16.9,
       46. , 39.7, 47. , 47.9, 47.6, 16.4, 40.1, 38. , 47.3, 46.5, 43.2,
       13.8, 23.1, 23.6, 15.9, 46.9, 23. , 23.5, 18.3, 45.1, 44. , 29.8,
       45.6, 46.1, 30. , 45.9, 22.4, 44.5, 45.5, 45.7, 46.2, 15.8, 46.8,
       41. , 47.4, 24. , 45. , 45.3, 40.4, 40.9, 42.1, 44.6, 20.8, 20.5,
       20. , 43.5, 44.3, 39. , 35.3, 35.8, 43.6, 44.1, 30.2, 30.7, 43.9,
       40. , 40.5, 41.5, 43.1,  5.7, 25.1, 25.6, 43. , 42.7, 15.7, 17.3,
       21.3, 18.1, 18.6, 42.8, 20.1, 20.6, 15.6, 42.3, 42.6, 42.2, 30.5,
       41.9, 41.7, 15.5, 41.3, 42.4, 42.9, 17.5, 34. , 41.1, 38.4, 38.9,
       41.2, 41.8, 23.8, 41.6, 35. , 22.1, 41.4, 35.9, 39.1, 15.4, 40.2,
       40.6, 40.7, 28.8, 40.3, 40.8, 35.5, 30.8, 15.3, 37.8, 38.2, 35.6,
       29. , 20.2, 20.7, 27.4, 21.1, 39.3, 39.8, 39.5, 22.6, 39.6, 18.4,
       39.2, 38.5, 24.8, 21. , 19.8, 24.3, 27. , 20

In [163]:
#Vehicle_Number_Of_Drivers_Assigned 99 drivers
df['Vehicle_Number_Of_Drivers_Assigned'].replace(99,np.NaN,inplace=True)
# # Vehicle_Number_Of_Drivers_Assigned_Mode=df['Vehicle_Number_Of_Drivers_Assigned'].mode()
df['Vehicle_Number_Of_Drivers_Assigned']=df['Vehicle_Number_Of_Drivers_Assigned'].fillna(9)
df['Vehicle_Number_Of_Drivers_Assigned'].unique()

array([1., 2., 3., 4., 5., 8., 7., 6.])

In [164]:
target=df['Loss_Amount']
df = df.drop(df.columns[58:], axis=1)
df

,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,Vehicle_Performance,Vehicle_Symbol,...,EEA_Multi_Auto_Policies_Indicator,EEA_Policy_Zip_Code_3,EEA_Policy_Tenure,EEA_Agency_Type,EEA_Packaged_Policy_Indicator,EEA_Full_Coverage_Indicator,EEA_Prior_Bodily_Injury_Limit,SYS_Renewed,SYS_New_Business,Annual_Premium
0,Standard,6,Direct Billed to Insured,Pre-paid,N,35,2004,BUI,Standard,11,...,N,430,22.7,Standard,N,Y,100-400,Y,N,320.12
2,Standard,6,Direct Billed to Insured,Pre-paid,N,17,2005,PON,Standard,10,...,N,435,47.2,Non-standard,N,Y,100-400,Y,N,613.74
3,Standard,6,Direct Billed to Insured,Pre-paid,N,17,2005,MER,Standard,12,...,Y,435,46.7,Non-standard,Y,Y,40-100,Y,N,541.66
4,Standard,6,Direct Billed to Insured,Pre-paid,N,17,2005,MER,Standard,12,...,Y,435,47.2,Non-standard,Y,Y,40-100,Y,N,541.66
5,Standard,6,Direct Billed to Insured,Pre-paid,N,27,1985,GMC,Standard,7,...,Y,430,16.5,Hybrid,Y,N,20-50,Y,N,164.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424424,Standard,6,Direct Billed to Insured,Pre-paid,N,31,2001,DOD,Standard,16,...,Y,434,0.0,Non-standard,Y,Y,20-50,N,Y,649.55
424425,Standard,12,Premium Finance,Pre-paid,N,36,2007,NSS,Standard,7,...,Y,425,0.0,Non-standard,Y,Y,100-200,Y,Y,541.26
424427,Standard,6,Direct Billed to Insured,Pre-paid,N,31,2000,NSS,Standard,8,...,Y,436,0.0,Hybrid,Y,N,100-200,Y,Y,117.13
424428,Standard,6,Direct Billed to Insured,Installment,N,35,1997,PON,Standard,8,...,Y,424,0.0,Preferred,N,Y,100-400,N,Y,118.21


In [165]:
le = LabelEncoder()
for col in df.columns:
    col = str(col)
    if str(df.loc[:,col].dtype) == 'object':
        le.fit(df.loc[:,col])
        df.loc[:,col] = le.transform(df.loc[:,col])
df

,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,Vehicle_Performance,Vehicle_Symbol,...,EEA_Multi_Auto_Policies_Indicator,EEA_Policy_Zip_Code_3,EEA_Policy_Tenure,EEA_Agency_Type,EEA_Packaged_Policy_Indicator,EEA_Full_Coverage_Indicator,EEA_Prior_Bodily_Injury_Limit,SYS_Renewed,SYS_New_Business,Annual_Premium
0,1,6,0,1,0,35,2004,25,4,11,...,0,68,22.7,3,0,1,1,1,0,320.12
2,1,6,0,1,0,17,2005,145,4,10,...,0,73,47.2,1,0,1,1,1,0,613.74
3,1,6,0,1,0,17,2005,123,4,12,...,1,73,46.7,1,1,1,5,1,0,541.66
4,1,6,0,1,0,17,2005,123,4,12,...,1,73,47.2,1,1,1,5,1,0,541.66
5,1,6,0,1,0,27,1985,84,4,7,...,1,68,16.5,0,1,0,2,1,0,164.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424424,1,6,0,1,0,31,2001,56,4,16,...,1,72,0.0,1,1,1,2,0,1,649.55
424425,1,12,1,1,0,36,2007,137,4,7,...,1,63,0.0,1,1,1,0,1,1,541.26
424427,1,6,0,1,0,31,2000,137,4,8,...,1,74,0.0,0,1,0,0,1,1,117.13
424428,1,6,0,0,0,35,1997,145,4,8,...,1,62,0.0,2,0,1,1,0,1,118.21


In [166]:
#df.dropna(subset = ["Annual_Premium"], inplace=True)
df.isnull().sum()

Policy_Company                              0
Policy_Installment_Term                     0
Policy_Billing_Code                         0
Policy_Method_Of_Payment                    0
Policy_Reinstatement_Fee_Indicator          0
Vehicle_Territory                           0
Vehicle_Make_Year                           0
Vehicle_Make_Description                    0
Vehicle_Performance                         0
Vehicle_Symbol                              0
Vehicle_Number_Of_Drivers_Assigned          0
Vehicle_Usage                               0
Vehicle_Miles_To_Work                       0
Vehicle_Days_Per_Week_Driven                0
Vehicle_Anti_Theft_Device                   0
Vehicle_Passive_Restraint                   0
Vehicle_Age_In_Years                        0
Vehicle_Med_Pay_Limit                       0
Vehicle_Bodily_Injury_Limit                 0
Vehicle_Physical_Damage_Limit               0
Vehicle_Comprehensive_Coverage_Indicator    0
Vehicle_Collision_Coverage_Indicat

In [167]:
df_copy=df.copy()

In [168]:
scaler = StandardScaler()
scaler.fit(df)
df_scaled=scaler.transform(df)
df = pd.DataFrame(df_scaled)
df

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.260796,-0.194625,-0.148395,1.023013,-0.367893,0.943944,0.989109,-1.343881,0.211284,-0.048291,...,-2.738221,-0.193375,2.648257,1.355151,-0.935114,0.801011,-0.575652,0.359098,-0.336556,-0.068619
1,0.260796,-0.194625,-0.148395,1.023013,-0.367893,-3.331673,1.126497,1.407333,0.211284,-0.298662,...,-2.738221,0.238775,6.472210,-0.391952,-0.935114,0.801011,-0.575652,0.359098,-0.336556,1.361103
2,0.260796,-0.194625,-0.148395,1.023013,-0.367893,-3.331673,1.126497,0.902943,0.211284,0.202080,...,0.365201,0.238775,6.394171,-0.391952,1.069388,0.801011,1.491787,0.359098,-0.336556,1.010124
3,0.260796,-0.194625,-0.148395,1.023013,-0.367893,-3.331673,1.126497,0.902943,0.211284,0.202080,...,0.365201,0.238775,6.472210,-0.391952,1.069388,0.801011,1.491787,0.359098,-0.336556,1.010124
4,0.260796,-0.194625,-0.148395,1.023013,-0.367893,-0.956330,-1.621271,0.008799,0.211284,-1.049775,...,0.365201,-0.193375,1.680562,-1.265503,1.069388,-1.248422,-0.058792,0.359098,-0.336556,-0.827352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403541,0.260796,-0.194625,-0.148395,1.023013,-0.367893,-0.006193,0.576944,-0.633151,0.211284,1.203564,...,0.365201,0.152345,-0.894754,-0.391952,1.069388,0.801011,-0.058792,-2.784753,2.971269,1.535472
403542,0.260796,5.138085,6.738789,1.023013,-0.367893,1.181478,1.401274,1.223918,0.211284,-1.049775,...,0.365201,-0.625524,-0.894754,-0.391952,1.069388,0.801011,-1.092512,0.359098,2.971269,1.008177
403543,0.260796,-0.194625,-0.148395,1.023013,-0.367893,-0.006193,0.439555,1.223918,0.211284,-0.799404,...,0.365201,0.325205,-0.894754,-1.265503,1.069388,-1.248422,-1.092512,0.359098,2.971269,-1.057037
403544,0.260796,-0.194625,-0.148395,-0.977505,-0.367893,0.943944,0.027390,1.407333,0.211284,-0.799404,...,0.365201,-0.711954,-0.894754,0.481600,-0.935114,0.801011,-0.575652,-2.784753,2.971269,-1.051778


In [169]:
pca = PCA(n_components=0.95)
principalComponents = pca.fit_transform(df)
principalDf = pd.DataFrame(data = principalComponents)
principalDf
df=principalDf.copy()
# pca = PCA(n_components=30).fit(df)
# principal_cols = pca.transform(df)
# df = pd.DataFrame(data = principal_cols)
df

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.021898,-3.108841,-1.158837,3.575376,-0.119242,2.214428,0.245883,1.978247,0.066509,0.825660,...,-0.389119,0.160094,0.575307,0.420190,-0.442733,0.512671,-1.064126,0.018924,-0.133564,0.604236
1,0.320501,-4.117123,-0.465979,4.858325,1.174517,2.260767,-0.960302,2.073748,1.337536,0.571509,...,1.853228,-1.002054,-0.810820,0.162367,-1.661612,0.816198,-1.268621,0.149280,-0.330381,1.026156
2,0.470361,-3.803084,-0.249355,3.348441,1.006890,3.466639,-1.729723,-1.246472,1.739382,1.181169,...,0.876638,-0.496573,0.085722,0.704878,-1.770166,-0.362288,-1.414089,0.416599,-0.026711,0.729304
3,0.444713,-3.849718,-0.241609,3.397896,1.008245,3.496114,-1.731320,-1.236319,1.747814,1.184279,...,0.890229,-0.485252,0.078105,0.704393,-1.790415,-0.378654,-1.433733,0.437350,-0.033758,0.731282
4,-2.317018,-0.369123,0.105726,-0.055863,-1.587042,-0.069546,0.437637,-0.442588,1.487044,0.557323,...,0.883872,0.333330,-0.942780,0.313192,-0.733578,0.340560,-0.025459,-0.889488,-0.144678,-0.392232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403541,4.037835,7.936314,2.059732,3.552443,2.150707,1.053484,-0.871199,-0.275925,-1.185191,-0.424566,...,7.553223,5.042420,8.345463,-0.284756,0.141301,5.776788,3.679310,3.805774,2.261303,-1.155774
403542,3.477832,-0.191341,-1.500166,-1.165852,5.937793,-3.275899,5.325033,-0.929476,2.526843,-0.872131,...,0.288852,2.064717,-1.074103,-0.181073,-0.529292,-0.338577,-0.347229,0.344098,-0.642523,2.285422
403543,1.362794,7.392149,0.997956,2.863489,-1.097914,-0.510724,1.254395,0.562845,-1.121561,-0.345888,...,2.792292,1.249410,2.821577,-0.352999,-1.564456,1.657828,1.808380,0.856409,-0.163248,1.073511
403544,-0.443953,-0.894339,-0.754904,-0.048778,1.851267,-1.726023,0.265799,0.116633,-3.319558,-1.092810,...,1.386537,-0.543547,-1.816114,-0.788280,-0.213705,0.383494,-0.185550,0.629773,-1.177798,0.687591


In [170]:
df['Loss_Amount'] = target
df['Annual_Premium'] = df_copy['Annual_Premium']
df.dropna(subset = ["Annual_Premium"], inplace=True)
df

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,Loss_Amount,Annual_Premium
0,0.021898,-3.108841,-1.158837,3.575376,-0.119242,2.214428,0.245883,1.978247,0.066509,0.825660,...,0.575307,0.420190,-0.442733,0.512671,-1.064126,0.018924,-0.133564,0.604236,0.0,320.12
2,0.470361,-3.803084,-0.249355,3.348441,1.006890,3.466639,-1.729723,-1.246472,1.739382,1.181169,...,0.085722,0.704878,-1.770166,-0.362288,-1.414089,0.416599,-0.026711,0.729304,0.0,613.74
3,0.444713,-3.849718,-0.241609,3.397896,1.008245,3.496114,-1.731320,-1.236319,1.747814,1.184279,...,0.078105,0.704393,-1.790415,-0.378654,-1.433733,0.437350,-0.033758,0.731282,0.0,541.66
4,-2.317018,-0.369123,0.105726,-0.055863,-1.587042,-0.069546,0.437637,-0.442588,1.487044,0.557323,...,-0.942780,0.313192,-0.733578,0.340560,-0.025459,-0.889488,-0.144678,-0.392232,0.0,541.66
5,1.168756,-4.388907,4.728072,2.738344,1.974252,1.080151,-0.124029,0.788132,-1.383441,1.641015,...,-0.981159,-0.087622,0.148477,-1.612573,-0.966439,0.574619,0.301955,-0.108659,0.0,164.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403541,4.037835,7.936314,2.059732,3.552443,2.150707,1.053484,-0.871199,-0.275925,-1.185191,-0.424566,...,8.345463,-0.284756,0.141301,5.776788,3.679310,3.805774,2.261303,-1.155774,0.0,167.48
403542,3.477832,-0.191341,-1.500166,-1.165852,5.937793,-3.275899,5.325033,-0.929476,2.526843,-0.872131,...,-1.074103,-0.181073,-0.529292,-0.338577,-0.347229,0.344098,-0.642523,2.285422,0.0,234.26
403543,1.362794,7.392149,0.997956,2.863489,-1.097914,-0.510724,1.254395,0.562845,-1.121561,-0.345888,...,2.821577,-0.352999,-1.564456,1.657828,1.808380,0.856409,-0.163248,1.073511,0.0,222.60
403544,-0.443953,-0.894339,-0.754904,-0.048778,1.851267,-1.726023,0.265799,0.116633,-3.319558,-1.092810,...,-1.816114,-0.788280,-0.213705,0.383494,-0.185550,0.629773,-1.177798,0.687591,0.0,526.82


In [171]:
df_copy2=df.copy()


In [172]:
def gen_portfolio(df):
    df = df.append(df, ignore_index=True)
    with_claims = []
    no_claims = []
    for i, r in enumerate(df['Loss_Amount']):
        if r > 0.0:
            with_claims.append(i)
        else:
            no_claims.append(i)
            
    random.shuffle(with_claims)
    random.shuffle(no_claims)
    portfolios = []
    i = 0
    while True:        
        size = 1000
#         loss_percent = np.random.choice(range(0, 5), p=[0.2]*5) * 5
#         if loss_percent == 0:
#             loss_percent = 2
                
        portfolio = []
        num_losses = 40
#         print(i,'-',size, loss_percent, num_losses)
        i += 1
        for i in range(num_losses):
            try:
                portfolio.append(df.iloc[with_claims.pop()])
            except Exception as e:
#                 print('breaking,', e)
                break

        for i in range(size-num_losses):
            try:
                portfolio.append(df.iloc[no_claims.pop()])
            except Exception as e:
#                 print('breaking,', e)
                break        

        if len(with_claims) == 0 or len(no_claims) == 0:                
            break
            
        random.shuffle(portfolio)
        portfolios.append(pd.DataFrame(portfolio, columns=df.columns))
    return portfolios

In [173]:
portfolio=gen_portfolio(df)
len(portfolio)
for p in portfolio:
    print(np.log(p['Loss_Amount'].sum()/p['Annual_Premium'].sum()))

-0.5895666133275669
-0.6854972711793195
-0.3855098161359671
-0.36414818304033014
-0.08538525449018854
-0.5074183228371728
-0.5681401845853367
-0.7066715984719261
-1.0907789464910165
-0.5608576614270064
-0.8455059739842232
-0.5365862101074623
-0.7791150237787097
-1.0730687859820258
-0.16647221795066958
-0.2911244270093065
-0.09977059889950055
-0.13734380248918687
-1.0812059705270773
-0.15029282240718417
-0.4798813229139053
-0.15545018853614828
-0.9716679701536535
-0.6241027690211399
-0.5023920837938114
-0.43591943716913323
-0.9769174580152105
-0.3645323178834316
-0.3881424150010444
-0.660619859920072
-0.8193339355518849
-0.4194977783861187
0.10358621234495849
-0.5543084068498849
-0.6385838193224982
-0.3145858255010383
-0.3801319966890064
-0.6040486138718039
-0.6904023324763072
-0.265110207220771
0.35012294022868384
-0.8427491540636992
-0.16588946845223204
-0.9539406306675998
-0.5245988945569875
-0.592172615666177
-0.9577863798168613
-0.041815753366289454
-0.25214901954129504
-0.71866935

In [174]:
def median_summarizer(df_t):    
    values = df_t.median()
    n_df = pd.DataFrame([], columns=df_t.columns)
    return n_df.append(values, ignore_index=True)

In [175]:
def summarize_portfolio(portfolio,convert_to_lr):
    summarized_portfolios = []
    flag = True
    for p in portfolio: 
        portfolios = median_summarizer(p)
        if convert_to_lr:
            portfolios['lr'] = np.log(p['Loss_Amount'].sum()/p['Annual_Premium'].sum())
            portfolios = portfolios.drop(['Loss_Amount', 'Annual_Premium'], axis=1)
        if flag:
                cols = portfolios.columns
                flag = False
        summarized_portfolios.append(portfolios)
    portoflio_attributes = pd.DataFrame([], columns=cols)
    for p in summarized_portfolios:
        portoflio_attributes = portoflio_attributes.append(p, ignore_index=True)
    return portoflio_attributes

In [176]:
training_portoflio_attributes=summarize_portfolio(portfolio,True)

In [181]:
training_portoflio_attributes


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,lr
32,0.434040,-0.493631,-0.464598,-0.101927,-0.078971,-0.107765,-0.180614,-0.019746,0.010149,0.135464,...,-0.055659,0.018933,-0.013055,0.103993,-0.008119,0.032544,-0.006458,0.019996,0.027423,0.103586
40,0.490970,-0.450980,-0.420176,-0.036682,0.030169,-0.076069,-0.244290,-0.081755,-0.041438,0.037519,...,-0.006123,-0.039709,-0.039387,0.172139,0.018611,-0.001311,-0.039211,0.024273,0.057195,0.350123
55,0.348481,-0.425594,-0.407517,-0.151928,0.037590,-0.056056,-0.191689,-0.112663,-0.047759,0.133928,...,-0.035887,-0.003023,-0.003103,0.085081,0.009387,0.007986,-0.010422,0.073936,0.008935,0.270794
60,0.458244,-0.554641,-0.349818,-0.065650,0.009460,-0.175117,-0.206816,-0.111653,0.041796,0.099647,...,-0.030073,-0.058078,0.010782,0.116968,0.004284,-0.024601,-0.025640,0.022845,-0.011136,0.030559
86,0.526112,-0.611703,-0.395683,-0.114732,-0.019917,-0.068418,-0.231518,-0.123346,-0.006283,0.123574,...,-0.024358,-0.053715,-0.021953,0.104318,0.022406,-0.045630,-0.060781,0.073009,-0.006753,0.215407
87,0.360794,-0.496189,-0.387794,-0.090319,0.021650,-0.093656,-0.151096,-0.114053,-0.007691,0.055153,...,-0.062521,-0.015204,0.010247,0.084055,0.052177,0.020811,-0.020395,-0.042896,-0.030184,0.097060
89,0.219062,-0.337320,-0.462081,-0.123395,-0.042285,-0.063895,-0.211768,-0.113889,0.041988,0.158816,...,-0.048113,-0.028368,-0.040029,0.131929,0.010573,0.026924,-0.024890,0.037969,0.024840,0.150513
92,0.512341,-0.516014,-0.500794,-0.132971,0.057663,-0.045604,-0.293607,-0.006485,0.022457,0.103280,...,-0.036413,-0.036535,-0.017491,0.072972,-0.041211,0.026945,-0.007214,0.011752,0.038591,0.026450
110,0.482607,-0.499162,-0.458758,-0.151732,0.090509,-0.056731,-0.213387,-0.081338,-0.070285,0.036616,...,-0.009593,-0.052745,-0.034039,0.067288,-0.002740,-0.030944,-0.019805,0.075865,-0.022541,0.244625
118,0.414965,-0.514118,-0.400471,-0.115182,0.113365,-0.122957,-0.220017,-0.022616,0.050326,0.118917,...,-0.043016,-0.021031,-0.001384,0.062354,0.044802,-0.018075,-0.035043,0.015230,0.081797,0.529979


In [178]:
import glob
path = '2020_Spring_testing_portfolios' # use your path
all_files = glob.glob(path + "/*.csv")
print(len(all_files))
portfolios = []
i = 0
for filename in all_files:
    i += 1
    testing_data = pd.read_csv(filename, index_col=None, header=0)
    testing_data = testing_data.drop(dropping_columns, axis=1)
    vehicles = [v[:3] for v in testing_data['Vehicle_Make_Description'].values]
    vehicles = pd.DataFrame(data = vehicles)
    testing_data['Vehicle_Make_Description'] = vehicles
    testing_data=testing_data.dropna()
    testing_data=testing_data[(testing_data['Vehicle_Anti_Theft_Device'] !="Unknown" )]
    testing_data=testing_data[(testing_data['Vehicle_Passive_Restraint'] !="Unknown" )]
    testing_data=testing_data[(testing_data['EEA_Policy_Zip_Code_3'] !="Unknown" )]
    testing_data=testing_data[(testing_data['Vehicle_Symbol'] !=-1 )]
    testing_data=testing_data[(testing_data['Vehicle_Bodily_Injury_Limit'] !='1M-1M  ' )]
    testing_data=testing_data[(testing_data['Driver_Maximum_Age']!=99)]
    testing_data=testing_data[(testing_data['Driver_Minimum_Age']!=99)]
    testing_data=testing_data[(testing_data.Driver_Maximum_Age != 0 )]
    testing_data=testing_data[(testing_data.Driver_Minimum_Age != 0 )]
    testing_data['Vehicle_Days_Per_Week_Driven'].replace(-1,np.NaN,inplace=True)
    Vehicle_Days_Per_Week_Driven_Mode=testing_data['Vehicle_Days_Per_Week_Driven'].mode()
    testing_data['Vehicle_Days_Per_Week_Driven']=testing_data['Vehicle_Days_Per_Week_Driven'].fillna(float(Vehicle_Days_Per_Week_Driven_Mode))
    testing_data['Vehicle_Miles_To_Work'].replace(-1,np.NaN,inplace=True)
    Vehicle_Miles_To_Work_Mean=testing_data['Vehicle_Miles_To_Work'].mean()
    a=int(Vehicle_Miles_To_Work_Mean)
    testing_data['Vehicle_Miles_To_Work']=testing_data['Vehicle_Miles_To_Work'].fillna(a)
    testing_data['Vehicle_Med_Pay_Limit'].replace(-1,np.NaN,inplace=True)
    Vehicle_Med_Pay_Limit=testing_data['Vehicle_Med_Pay_Limit'].mean()
    testing_data['Vehicle_Med_Pay_Limit']=testing_data['Vehicle_Med_Pay_Limit'].fillna(int(Vehicle_Med_Pay_Limit))
    testing_data['Vehicle_Physical_Damage_Limit'].replace(-1,np.NaN,inplace=True)
    Vehicle_Physical_Damage_Limit_Mean=testing_data['Vehicle_Physical_Damage_Limit'].mean()
    testing_data['Vehicle_Physical_Damage_Limit']=testing_data['Vehicle_Physical_Damage_Limit'].fillna(int(Vehicle_Physical_Damage_Limit_Mean))
    testing_data['Vehicle_Number_Of_Drivers_Assigned'].replace(99,np.NaN,inplace=True)
    Vehicle_Number_Of_Drivers_Assigned_Mode=testing_data['Vehicle_Number_Of_Drivers_Assigned'].mean()
    testing_data['Vehicle_Number_Of_Drivers_Assigned']=testing_data['Vehicle_Number_Of_Drivers_Assigned'].fillna(int(Vehicle_Number_Of_Drivers_Assigned_Mode))
    testing_data['Vehicle_Collision_Coverage_Deductible'].replace(-1,np.NaN,inplace=True)
    Vehicle_Collision_Coverage_Deductible_Mean=testing_data['Vehicle_Collision_Coverage_Deductible'].mean()
    testing_data['Vehicle_Collision_Coverage_Deductible']=testing_data['Vehicle_Collision_Coverage_Deductible'].fillna(int(Vehicle_Collision_Coverage_Deductible_Mean))
    testing_data['EEA_Policy_Tenure'].replace(-1,np.NaN,inplace=True)
    EEA_Policy_Tenure_Mean=testing_data['EEA_Policy_Tenure'].mean()
    testing_data['EEA_Policy_Tenure']=testing_data['EEA_Policy_Tenure'].fillna(round(EEA_Policy_Tenure_Mean,1))
    testing_data['Vehicle_Number_Of_Drivers_Assigned'].replace(99,np.NaN,inplace=True)
    testing_data['Vehicle_Number_Of_Drivers_Assigned']=testing_data['Vehicle_Number_Of_Drivers_Assigned'].fillna(9)
    le = LabelEncoder()
    for col in testing_data.columns:
        col = str(col)
        if str(testing_data.loc[:,col].dtype) == 'object':
            le.fit(testing_data.loc[:,col])
            testing_data.loc[:,col] = le.transform(testing_data.loc[:,col])
    testing_data_scaled = scaler.transform(testing_data)
    testing_data = pd.DataFrame(testing_data_scaled)
    testing_data_pca = pca.transform(testing_data)
    testing_data = pd.DataFrame(data = testing_data_pca)
    portfolios.append(testing_data)

330


In [182]:
testing_portfolios_attributes=summarize_portfolio(portfolios,False)

In [183]:
testing_portfolios_attributes


,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.415639,-0.609132,-0.461352,0.142086,-0.025785,-0.884946,-0.144793,-0.167756,0.195834,-0.008193,...,-1.450147,-2.159291,0.951390,0.340320,0.214171,0.147668,-0.234434,0.029309,0.158554,-0.052456
1,0.544122,-0.565094,-0.566886,0.223772,-0.013655,-0.884011,-0.180244,-0.215707,0.123299,0.003265,...,-1.394567,-2.136160,0.919090,0.292619,0.227671,0.165278,-0.243901,0.031324,0.124699,-0.081516
2,0.228740,-0.443253,-0.485343,0.141143,-0.051114,-0.957238,-0.148102,-0.171321,0.147225,0.002632,...,-1.359390,-2.068137,0.951879,0.375003,0.225463,0.129228,-0.242433,0.062143,0.234420,-0.046034
3,0.450934,-0.702176,-0.486118,0.385100,-0.069982,-0.966124,-0.171310,-0.054057,0.255454,-0.063114,...,-1.365336,-2.106270,0.934506,0.297672,0.225576,0.162776,-0.219248,0.022200,0.230332,-0.022083
4,0.465863,-0.545438,-0.518676,0.158260,0.103952,-0.811413,-0.204409,-0.240930,0.153531,0.006297,...,-1.365763,-2.098162,0.901284,0.324631,0.235998,0.158312,-0.165158,-0.001562,0.213216,-0.098076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,0.480554,-0.552611,-0.435762,0.292904,0.013241,-1.018607,-0.143310,-0.111292,0.046120,-0.034243,...,-1.397184,-2.133352,0.925788,0.321775,0.207607,0.154697,-0.159010,0.048708,0.147742,-0.135174
326,0.550142,-0.509718,-0.557635,0.042833,0.023368,-0.883218,-0.165734,-0.292894,0.081475,-0.089096,...,-1.389748,-2.101781,0.899606,0.308566,0.247569,0.201584,-0.186869,0.007738,0.080053,-0.098891
327,0.387228,-0.396196,-0.468440,0.095245,0.044579,-0.678603,-0.165395,-0.258088,0.135400,-0.010355,...,-1.427018,-2.095825,0.947391,0.292796,0.249825,0.148993,-0.269306,0.042274,0.181925,-0.051866
328,0.430806,-0.454079,-0.470059,0.032911,-0.089594,-0.735164,-0.117712,-0.311770,0.042219,0.001096,...,-1.412154,-2.085050,0.924200,0.319873,0.214091,0.120505,-0.224619,0.023955,0.171362,-0.130153


In [204]:
def getOutput(predictions,filename):
    path = "2020_Spring_testing_portfolios" # use your path
    output_file=filename
    all_files = glob.glob(path + "/*.csv")
    print(f'found {len(all_files)} files.')
    files = []
    for f in all_files:
        t = f[36:]
        t = t[:-4]
        files.append(t)
    def sort_file(item):
        return int(item.split('_')[1])
#     files = sorted(files, key=sort_file)
    assert len(predictions) == len(files)
    output = []
    print(f'constructing output for {len(predictions)} predictions')
    for i in range(len(predictions)):
        pred = predictions[i]
        output.append((files[i], pred))

    if output_file is not None:
        op_pd = pd.DataFrame(output, columns=['ID','ln_LR'])
        op_pd.to_csv(output_file + '.csv', index=False)

In [205]:
training=training_portoflio_attributes.copy()
loss_ratio=training['lr']
training=training.drop('lr',axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(training.values,loss_ratio.values, test_size=0.25, random_state=42)
X_test


array([[ 0.36139087, -0.55498044, -0.39117773, ..., -0.05480852,
         0.03837832, -0.03289588],
       [ 0.35299325, -0.47729088, -0.37486547, ...,  0.03088184,
        -0.02830436,  0.03737897],
       [ 0.179729  , -0.48140818, -0.48153481, ..., -0.02318086,
         0.02560494, -0.0323139 ],
       ...,
       [ 0.37201765, -0.48157263, -0.41169049, ...,  0.04168557,
         0.02099176,  0.01625383],
       [ 0.10719451, -0.52217829, -0.36882322, ...,  0.03571321,
        -0.00310735, -0.00426071],
       [ 0.40599015, -0.4338496 , -0.50442503, ..., -0.02533435,
         0.01796915,  0.01623745]])

In [206]:
mlp = MLPRegressor(hidden_layer_sizes=(100, ), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10)
mlp.fit(X_train,y_train)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [232]:
predictions=mlp.predict(testing_portfolios_attributes)
mlp.score(X_test,y_test)

-0.018906681164584382

In [188]:
filename="MLP"
getOutput(predictions,filename)

found 330 files.
constructing output for 330 predictions


In [231]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import cross_val_score, GridSearchCV
# create regressor object 
regressor = RandomForestRegressor(n_estimators = 500, random_state = 0)
# fit the regressor with x and y data 
regressor.fit(X_train ,y_train)
RFR_pred=regressor.predict(testing_portfolios_attributes.values)
print(regressor.score(X_test,y_test))
RFR_pred

-0.0624414927159036


array([-0.0283653 , -0.06881671, -0.03372892, -0.06353149,  0.04080164,
        0.03099585, -0.0493638 , -0.22433275, -0.17695903, -0.06036517,
       -0.057777  , -0.21172181, -0.04662683, -0.02818425, -0.01972703,
        0.06408554, -0.08635956,  0.00182391, -0.02543756, -0.07660514,
       -0.20065919,  0.21015093, -0.07832766, -0.02281505,  0.14342915,
       -0.0712092 , -0.07529747, -0.01411464, -0.04745076, -0.02959367,
       -0.07457983, -0.03734981,  0.12302777, -0.03107767,  0.09323618,
       -0.11783134, -0.00278885, -0.04939299,  0.06543907, -0.01189101,
        0.19240094, -0.19388953, -0.06037669, -0.14802426, -0.03196822,
       -0.12273199, -0.02810553,  0.12574914, -0.05421742,  0.14942453,
       -0.05315547, -0.07355779,  0.08216913, -0.34933927,  0.08859151,
       -0.35145276, -0.03789453, -0.01057272, -0.07873946, -0.2326254 ,
        0.0056985 , -0.00427136,  0.03423096, -0.0372645 , -0.05739624,
       -0.0344157 , -0.0320592 , -0.05432606, -0.05563805, -0.05

In [239]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
gsc = GridSearchCV( estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (500, 550),
        },cv=5, scoring='neg_mean_squared_error', verbose=0,n_jobs=-1)
    
grid_result = gsc.fit(X_train, y_train)
best_params = grid_result.best_params_

rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"], random_state=False, verbose=False)

scores = cross_val_score(rfr, X_train, y_train, cv=10, scoring='neg_mean_absolute_error')
scores

array([-0.2901818 , -0.3679626 , -0.2960869 , -0.2904251 , -0.27688487,
       -0.30447941, -0.30066193, -0.31600675, -0.30688728, -0.26781618])

In [237]:
rfr.fit(X_train,y_train)
score=rfr.score(X_test,y_test)
print(score)

array([-0.12642111, -0.16964713, -0.13043977, -0.1569217 , -0.13851018,
       -0.08760016, -0.15300115, -0.37176595, -0.23366501, -0.14719299,
       -0.22355722, -0.35757043, -0.14282523, -0.13522006, -0.1499376 ,
       -0.06969157, -0.22575395, -0.15610124, -0.13201971, -0.15781344,
       -0.2627925 ,  0.04816006, -0.20843096, -0.11219144,  0.06173036,
       -0.15114732, -0.20610468, -0.11999245, -0.21744   , -0.15530558,
       -0.22624716, -0.11124089,  0.02695705, -0.12291516,  0.07842512,
       -0.23973066, -0.10298621, -0.15630738, -0.08076214, -0.15495016,
        0.05820797, -0.23366501, -0.15646636, -0.23225277, -0.14395443,
       -0.17468782, -0.12057915,  0.01160127, -0.185146  , -0.00955464,
       -0.21218771, -0.22575395, -0.05460966, -0.55103881, -0.07178271,
       -0.55199199, -0.15271624, -0.11904362, -0.22575395, -0.42418782,
       -0.08233269, -0.16982171, -0.12821856, -0.12750793, -0.16180572,
       -0.13188337, -0.12772746, -0.17014034, -0.15781344, -0.14

In [210]:
l1_cv = LassoCV(cv=X_train.shape[0],max_iter=1000, alphas=np.linspace(0.1,1000,100))
l1_cv.fit(X_train, y_train)

In [191]:
l1_cv_pred=l1_cv.predict(testing_portfolios_attributes)

In [192]:
l1_cv_pred

array([-0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53612131,
       -0.53612131, -0.53612131, -0.53612131, -0.53612131, -0.53

In [193]:
l2_cv = RidgeCV(cv=None, store_cv_values=True, alphas=np.linspace(0.1,1000,100))
l2_cv.fit(X_train, y_train)

RidgeCV(alphas=array([1.000e-01, 1.020e+01, 2.030e+01, 3.040e+01, 4.050e+01, 5.060e+01,
       6.070e+01, 7.080e+01, 8.090e+01, 9.100e+01, 1.011e+02, 1.112e+02,
       1.213e+02, 1.314e+02, 1.415e+02, 1.516e+02, 1.617e+02, 1.718e+02,
       1.819e+02, 1.920e+02, 2.021e+02, 2.122e+02, 2.223e+02, 2.324e+02,
       2.425e+02, 2.526e+02, 2.627e+02, 2.728e+02, 2.829e+02, 2.930e+02,
       3.031e+02, 3.132e+02, 3.233e+0...
       7.273e+02, 7.374e+02, 7.475e+02, 7.576e+02, 7.677e+02, 7.778e+02,
       7.879e+02, 7.980e+02, 8.081e+02, 8.182e+02, 8.283e+02, 8.384e+02,
       8.485e+02, 8.586e+02, 8.687e+02, 8.788e+02, 8.889e+02, 8.990e+02,
       9.091e+02, 9.192e+02, 9.293e+02, 9.394e+02, 9.495e+02, 9.596e+02,
       9.697e+02, 9.798e+02, 9.899e+02, 1.000e+03]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
        scoring=None, store_cv_values=True)

In [194]:
l2_cv_pred=l2_cv.predict(testing_portfolios_attributes)

In [195]:
l2_cv_pred

array([-0.5425336 , -0.54323993, -0.54227902, -0.54228157, -0.54264124,
       -0.54224162, -0.54291478, -0.54411299, -0.54303269, -0.54318764,
       -0.54372272, -0.5425323 , -0.54174653, -0.54260893, -0.54260661,
       -0.54237307, -0.54367206, -0.54309064, -0.54266348, -0.5434486 ,
       -0.54241986, -0.54238349, -0.5436184 , -0.54246025, -0.54322943,
       -0.54336363, -0.54304285, -0.54199889, -0.54113197, -0.54282938,
       -0.54313399, -0.54319366, -0.54256826, -0.54252505, -0.54244343,
       -0.54421037, -0.54229163, -0.54241092, -0.54172509, -0.54223226,
       -0.54153291, -0.54282917, -0.54309117, -0.54174178, -0.54223932,
       -0.54208413, -0.54220162, -0.54234937, -0.54321516, -0.54325026,
       -0.54342634, -0.54389622, -0.54364699, -0.54246262, -0.54416703,
       -0.54418737, -0.54415125, -0.54263573, -0.54380014, -0.54323047,
       -0.54197021, -0.54297526, -0.54275722, -0.54214781, -0.54286182,
       -0.54244806, -0.5424152 , -0.54263301, -0.54284588, -0.54